In [66]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    def __init__(self, generating_pol, defining_set):
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = generating_pol.base_ring().prime_subfield()
        
        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        self._field = generating_pol.base_ring().prime_subfield()
        self._dimension = len(defining_set) - generating_pol.degree()
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
    def get_generating_pol(self):
        return self._generating_pol
    
    def get_defining_set(self):
        return self._defining_set
    
    def parity_check_matrix(self):
        g = self._generating_pol
        L = self._defining_set
        n = self._length
        d = g.degree()
        
        aux = vector([0 for i in range(n)])  # auxiliary vector of n columns
        H = matrix(aux)

        for i in range(0, d):
            elem = g(L[0]).inverse_of_unit() * L[0]**i   # first row
            c = vector(elem).column()
            aux = matrix(c)
            
            for j in range(1,n):
                elem = g(L[j]).inverse_of_unit() * L[j]**i
                c = vector(elem).column()
                aux = aux.augment(c)
            
            H = H.stack(aux)

        H = H.delete_rows([0])   # delete auxiliary vector
        
        return H
    
    def _repr_(self):
        return "[{}, {}] Goppa code".format(self.length(), self.dimension())

class GoppaEncoder(Encoder):
    def __init__(self, code):
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        return "Encoder for {}".format(self.code())
    
    def generator_matrix(self):
        H = self.code().parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        return G
    
    def encode (self, m):
        return m * generator_matrix()

class GoppaDecoder(Decoder):
    def __init__(self, code):
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
        
        print("TODO")
        
        g = self.code().get_generating_pol()
        L = self.code().get_defining_set()
        
        syndrome = list()
        
        for i in range(len(L)):
            syndrome.append((g.parent().gen() - L[i]).inverse_mod(g))
            
        self._syndrome = syndrome
    
    def get_syndrome(self):
        return self._syndrome

Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder
Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

In [31]:
F = GF(2^3)
R.<x> = F[]
g = x^2 + x + 1
L = [a for a in F.list() if g(a) != 0]
C = Goppa(g, L)
C

[8, 6] Goppa code

In [ ]:
H = C.parity_check_matrix()

In [39]:
g.base_ring().gen()

z3

In [25]:
C

[8, 6] Goppa code

In [42]:
g.base_ring()

Finite Field in z3 of size 2^3

In [ ]:
F = GF(2^3)
R.<x> = F[]
g = (x^2 + x + 1)^2
L = [a for a in F.list() if g(a) != 0]
E = GoppaEncoder(C)
E

In [ ]:
G = E.generator_matrix()

In [ ]:
H * transpose(G)

In [ ]:
G * transpose(H)

# ALGORITMO DE EUCLIDES

In [ ]:
def euclides (a, b):
    # TODO

# **ALGORITMO DE SUGIYAMA**

1. Calcular el síndrome $S(x)$.
2. Sean $r_{-1}(x) = g(x)$, $r_0(x) = S(x)$, $U_{-1}(x) = 0$ y $U_0(x) = 1$.
3. Buscar $q_i(x)$ y $r_i(x)$ aplicando el algoritmo de Euclides para encontrar el máximo común divisor de $r_{-1}(x)$ y $r_0(x)$ para $i = 1,..., k$, hasta que $k$ cumpla que $gr(r_{k-1}(x)) \geq t$ y $gr(r_k(x)) < t$:

$$r_{i-2}(x) = r_{i-1}(x) q_i(x) + r_i(x), \qquad gr(r_i(x)) < gr(r_{i-1})(x)$$
    
4. Calcular $U_k(x)$, donde
    
$$U_i(x) = q_i(x) U_{i-1}(x) + U_{i-2}(x)$$

5. La solución viene dada por:
$$\eta(x) = (-1)^k \delta r_k(x)$$
$$\sigma(x) = \delta U_k(x)$$

In [ ]:
def sugiyama(code):
    i = 1
    
    # Paso 1
    S = code.get_syndrome()
    
    # Paso 2
    r_prev = code.get_generating_pol()
    r_i = S
    U_prev = 0
    U_i = 1
    
    # Paso 3 y 4
    while r_prev.degree() < t and r_i.degree() >= t:
        (q_i, r_i) = euclides(r_prev, r_i)
        U_i = q_i * U_i + U_prev
        r_prev = r_i
        U_prev = U_i
        i += 1
        
    # Paso 5
    eta = (-1)^i * r_i
    sigma = U_i
    
    return eta, sigma